In [253]:
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/dataset91/data.py", dst = "../working/data.py")
copyfile(src = "../input/dataset91/utils.py", dst = "../working/utils.py")
copyfile(src = "../input/dataset91/training.py", dst = "../working/training.py")
copyfile(src = "../input/dataset91/layers.py", dst = "../working/layers.py")
#copyfile(src = "../input/notes1/weights2.npy", dst = "../working/weights2.npy")
# import all our functions
#from data import load_data

'../working/layers.py'

In [254]:
# Estimate the GPS clusters
print("Estimating clusters...")
#clusters = get_clusters(data.train_labels)
n_epochs=100
batch_size=200
save_prefix='mymodel'
config=3


Estimating clusters...


In [255]:
#import pickle
!pip install pickle5
import csv
import calendar
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import  MultipleLocator, FormatStrFormatter
from scipy.interpolate import splrep
from IPython.core.display import display_html
from keras.models import load_model
from utils import np_haversine, density_map, get_clusters, plot_embeddings
from data import load_data
#from training import start_new_session, process_features, create_model

# Display plots inline
%matplotlib inline

# Fix random seed for reproducibility
np.random.seed(42)

In [256]:
def start_new_session():
    """
    Starts a new Tensorflow session.
    """
    
    # Make sure the session only uses the GPU memory that it actually needs
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    
    session = tf.compat.v1.Session(config=config, graph=tf.compat.v1.get_default_graph())
    tf.compat.v1.keras.backend.set_session(session)

In [257]:
clusters_cache = '/kaggle/input/dataset91/clusters.npy'
clusters=np.load(clusters_cache, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [258]:
file='/kaggle/input/dataset91/train0.npy'
train0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train1.npy'
train1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train2.npy'
train2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train3.npy'
train3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train4.npy'
train4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train5.npy'
train5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train6.npy'
train6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
train=[train0,train1,train2,train3,train4,train5,train6]

In [259]:
file='/kaggle/input/dataset91/validation0.npy'
validation0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation1.npy'
validation1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation2.npy'
validation2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation3.npy'
validation3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation4.npy'
validation4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation5.npy'
validation5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation6.npy'
validation6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
validation=[validation0,validation1,validation2,validation3,validation4,validation5,validation6]

In [260]:
train=train[6]
validation=validation[6]

In [261]:
!pip3 install pickle5
import pickle5 as pickle
metadata_cache = '/kaggle/input/dataset91/metadata.pickle'
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)
tl_cache = '/kaggle/input/dataset91/train-labels.npy'
vl_cache = '/kaggle/input/dataset91/validation-labels.npy'
train_labels=np.load(tl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
validation_labels=np.load(vl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')

In [262]:
import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Reshape, Activation, Dropout
from keras.layers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from utils import tf_haversine
from data import load_data
from utils import get_clusters
from keras.models import Model

from tensorflow.compat.v1.keras.layers import LSTM   # CuDNNLSTM
from tensorflow.compat.v1.keras.layers import GRU
import sys
sys.path.append('..')  # add parent directory to Python path for layers.py access
from layers import Attention, SelfAttention

def create_model(metadata,clusters,config):
    """
    Creates all the layers for our neural network model.
    """
      
    # Arbitrary dimension for all embeddings
    embedding_dim = 10

    
    
    # GPS coordinates (5 first lat/long and 5 latest lat/long, therefore 20 values)
    coords = Sequential()
    coords.add(Dense(1, input_dim=20,input_shape=(20,1)))

    # Merge all the inputs into a single input layer
    mergedOut = coords.output
    
    #encoder_output, hidden_state, cell_state = LSTM(50, activation='tanh',input_shape=(None,None, 1),
    #                                                return_sequences=True,return_state=True)(mergedOut)
    #attention_input = [encoder_output, hidden_state]
    gru_layer=tf.keras.layers.GRU(200, return_sequences=True,return_state=True)
    if config != 0:
        #encoder_output, hidden_state, cell_state = LSTM(units=200,
        #                                                 return_sequences=True
        #                                                ,return_state=True#,input_shape=(50,1)
        #                                               )(mergedOut)
        encoder_output, hidden_state, cell_state = Bidirectional(gru_layer,
                                                                      merge_mode="sum")(mergedOut)
        attention_input = [encoder_output, hidden_state]
    else:
        encoder_output = LSTM(units=64)(mergedOut)

    # Optional Attention Mechanisms
    if config == 1:
        encoder_output, attention_weights = SelfAttention(size=64,
                                                      num_hops=10,
                                                      use_penalization=False)(encoder_output)
    elif config == 2:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='global')(attention_input)
        encoder_output = Flatten()(encoder_output)
    elif config == 3:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='local-m',
                                                  window_width=10,
                                                  score_function='general')(attention_input)
        encoder_output = Flatten()(encoder_output)
        
    
    #encoder_output=Dense(1)(encoder_output)
    # Determine cluster probabilities using softmax
    mergedOut=Dense(len(clusters))(encoder_output)
    mergedOut=Activation('softmax')(mergedOut)

    # Final activation layer: calculate the destination as the weighted mean of cluster coordinates
    cast_clusters = K.cast_to_floatx(clusters)
    def destination(probabilities):
        return tf.matmul(probabilities, cast_clusters)
    mergedOut=Activation(destination)(mergedOut)

    newModel = Model([coords.input],mergedOut)
    #use lists if you want more than one input or output  
    
    # Compile the model
    optimizer = SGD(lr=0.1, momentum=0.9, clipvalue=1.)  # Use `clipvalue` to prevent exploding gradients

    newModel.compile(loss=tf_haversine, optimizer=optimizer)
    
    return newModel

In [263]:
model=create_model(metadata,clusters,config)

In [264]:
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dense_28_input (InputLayer)     [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
dense_28 (Dense)                (None, 20, 1)        2           dense_28_input[0][0]             
__________________________________________________________________________________________________
bidirectional_14 (Bidirectional [(None, 20, 200), (N 243600      dense_28[0][0]                   
__________________________________________________________________________________________________
attention_14 (Attention)        ((None, 10, 200), (N 40000       bidirectional_14[0][0]           
                                                                 bidirectional_14[0][1]    

In [265]:
e=[]
f=[]
g=[]
for i in range(91):
    f.append(0)
    e.append(0)
    g.append(0)

In [266]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(e)
        if z>len(g):
            a=model.history.history['val_loss'][z-len(g)-1]
            b=model.history.history['loss'][z-len(g)-1]
            c=float(a)-float(b)
            if c<0:
                print("Sebelumnya Underfitting")
            else:
                print("Sebelumnya Overfitting")
        e.append(0)
            

In [267]:
class CustCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(f)
        file_path="../working/Newweights"+str(z)
        weight=model.get_weights()
        np.save(file_path, weight)
        f.append(0)

In [268]:
#file='../input/latbigru91gpslocm/Newweights'+str(len(g)-1)+'.npy'
file='./Newweights'+str(len(g)-1)+'.npy'
#file='../input/dataset91/Newweights'+str(len(g)-1)+'.npy'
b=np.load(file, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
model.set_weights(b)

In [269]:
file='../input/dataset91/train-labels.npy'
train_labels=np.load(file, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
file='../input/dataset91/validation-labels.npy'
validation_labels=np.load(file, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')

In [270]:
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.callbacks import EarlyStopping
 
callbacks = []
if save_prefix is not None:
        # Save the model's intermediary weights to disk after each epoch
    file_path="cache/%s-{epoch:03d}-{val_loss:.4f}.hdf5" % save_prefix
    checkpoint = ModelCheckpoint(file_path,monitor='val_loss',mode='auto',save_weights_only=True,verbose=0)
    #checkpoint = weight_save(model.get_weights(),b)
    callbacks.append(checkpoint)
    #callbacks.append(ModelCheckpoint(file_path, monitor='val_loss', mode='min', save_weights_only=True, verbose=1))
    #callbacks.append(ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=False, mode='auto'))
    #model.fit(X_train,y_train,batch_size=batch_size,nb_epoch=nb_epoch,callbacks=[weight_save_callback])
#g.append(0)
#epoch>1

#first epoch
#saver = weight_save(model.get_weights(),b)   
#es = EarlyStopping(monitor='val_loss', patience=0, verbose=1)
arc='LATBiGRUnew91-b200-'+str(len(g))+'.log'
csv_logger = CSVLogger(arc, separator=',', append=False)
print("Creating model...")
start_new_session()
print("Train model...")
history=model.fit(train, train_labels,
        initial_epoch=len(g),epochs=100, batch_size=batch_size,
        validation_data=(validation, validation_labels)
        ,callbacks=[csv_logger,CustCallback(),MyCallback()])


Creating model...
Train model...


2021-12-17 11:20:50.900096: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 11:20:50.900853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 11:20:50.901404: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 11:20:50.902041: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 11:20:50.902590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Epoch 92/100


2021-12-17 11:20:53.841447: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "11000" } environment { key: "cudnn" value: "8005" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16152002560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


7397/7400 [============================>.] - ETA: 0s - loss: 1.3951

2021-12-17 11:22:20.727761: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "11000" } environment { key: "cudnn" value: "8005" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16152002560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


7400/7400 [==============================] - 94s 12ms/step - loss: 1.3951 - val_loss: 1.4323
Epoch 93/100
7400/7400 [==============================] - 91s 12ms/step - loss: 1.3916 - val_loss: 1.4572
Sebelumnya Overfitting
Epoch 94/100
7400/7400 [==============================] - 91s 12ms/step - loss: 1.3914 - val_loss: 1.4121
Sebelumnya Overfitting
Epoch 95/100
7400/7400 [==============================] - 89s 12ms/step - loss: 1.3918 - val_loss: 1.4138
Sebelumnya Overfitting
Epoch 96/100
7400/7400 [==============================] - 89s 12ms/step - loss: 1.3926 - val_loss: 1.4362
Sebelumnya Overfitting
Epoch 97/100
7400/7400 [==============================] - 89s 12ms/step - loss: 1.3924 - val_loss: 1.4234
Sebelumnya Overfitting
Epoch 98/100
7400/7400 [==============================] - 89s 12ms/step - loss: 1.3873 - val_loss: 1.4330
Sebelumnya Overfitting
Epoch 99/100
7400/7400 [==============================] - 89s 12ms/step - loss: 1.3907 - val_loss: 1.4293
Sebelumnya Overfitting
Epoc

In [ ]:
t=0
while(t<1):
    t=t